In [1]:
import os
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from scipy.special import expit

In [67]:
# Cargo el excel de la EPH
eph_todo = pd.read_csv("eph_panel.csv")

C:\Users\felip\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (102,179,182,187,207,209) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [68]:
def limpiar_eph(eph):

    '''
        función para limpiar la eph
        input:
            eph: Variable que tenga almacenada la eph sin limpiar

        output:
            eph: Variable que tiene almacenada la eph limpia
    '''
    
    eph.dropna(thresh = 1, inplace = True) # Si tiene todos NA, la fila se va
    
    eph.dropna(thresh = 1, axis = "columns", inplace = True) # Si la columna tiene todos NA, la columna se va
    
    try:
        # Si no lo defino como uno me dropea un montón de filas que tienen NaN
        eph.PP08D1 = eph.PP08D1.fillna(0)
        # dropeo observaciones con valores que no tienen sentido. CH06 = edad, PP08D1, P21, P47T, ITF y IPCF son todas variables
        # de ingresos, no pueden ser menores a 0
        eph.drop(eph[(eph["CH06"] < 0) | (eph["PP08D1"] < 0) | (eph["P21"] < 0) | \
                           (eph["P47T"] < 0) | (eph["ITF"] < 0) | (eph["IPCF"] < 0)].index, inplace = True)
        eph.fillna(0, inplace = True) # Relleno todos con 0 los missing de las columnas que representan subcategorías en
        #las que solo respondieron algunos. 
        eph.reset_index(inplace = True, drop = True)  
    except:
        print("No se encontró alguna columna, revisar")   

In [69]:
columnas_ing = ["AGLOMERADO","COMPONENTE","MAS_500","CH04","CH06","CH08","PP02E","PP02I","PP3E_TOT","PP3F_TOT",
"CH09","CH10","CH11","CH12","CH13","CH15","CH07","CH16","NIVEL_ED","ESTADO","CAT_OCUP","CAT_INAC","PP04B3_ANO",
"PP03G","PP03I","INTENSI","PP11T","PP11S","PP11O","PP11N","PP11L1","PP10D","PP10A","PP10C","PP07H","PP07G4","PP11A",
"PP10E","PP09C","PP07K","PP07G_59","PP07G2","PP07G1","PP07C","PP07A","PP05H","PP04G","IV2","IV11","IV12_3","IV1", 
"TRIMESTRE","CODUSU","ANO4","PONDERA","II7","II8","II9","V5","V17","IV3","V4","IV6","IV7","IV8","IV9","V3","V6","V8",
"IV10","IV11","II4_1","II4_2","II4_3","V1","V2","V9","V10","V13","V14","V15","V18","IX_TOT","IX_MEN10","ITF","DECIFR",
"ADECIFR","IPCF","DECCFR","ADECCFR","PONDIH","PP08D1","P21","DECOCUR","TOT_P12","P47T"]

columnas = ["AGLOMERADO","COMPONENTE","MAS_500","CH04","CH06","CH08","PP02E","PP02I","PP3E_TOT","PP3F_TOT",
"CH09","CH10","CH11","CH12","CH13","CH15","CH07","CH16","NIVEL_ED","ESTADO","CAT_OCUP","CAT_INAC","PP04B3_ANO",
"PP03G","PP03I","INTENSI","PP11T","PP11S","PP11O","PP11N","PP11L1","PP10D","PP10A","PP10C","PP07H","PP07G4","PP11A",
"PP10E","PP09C","PP07K","PP07G_59","PP07G2","PP07G1","PP07C","PP07A","PP05H","PP04G","IV2","IV11","IV12_3","IV1", 
"TRIMESTRE","CODUSU","ANO4","PONDERA","II7","II8","II9","V5","V17","IV3","V4","IV6","IV7","IV8","IV9","V3","V6","V8",
"IV10","IV11","II4_1","II4_2","II4_3","V1","V2","V9","V10","V13","V14","V15","V18","IX_TOT","IX_MEN10"]

In [70]:
eph = eph_todo[columnas_ing]

In [71]:
limpiar_eph(eph)

<ipython-input-68-88bca93152a5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eph.dropna(thresh = 1, inplace = True) # Si tiene todos NA, la fila se va
<ipython-input-68-88bca93152a5>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eph.dropna(thresh = 1, axis = "columns", inplace = True) # Si la columna tiene todos NA, la columna se va
C:\Users\felip\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

In [72]:
eph

,AGLOMERADO,COMPONENTE,MAS_500,CH04,CH06,CH08,PP02E,PP02I,PP3E_TOT,PP3F_TOT,...,ADECIFR,IPCF,DECCFR,ADECCFR,PONDIH,PP08D1,P21,DECOCUR,TOT_P12,P47T
0,26,1,N,1,71,1,0,2,0.0,0.0,...,6.0,7500.0,7.0,8.0,176,0.0,0,0,0,10000.0
1,26,1,N,1,72,1,0,2,0.0,0.0,...,6.0,9450.0,7.0,8.0,186,0.0,0,0,0,14000.0
2,26,1,N,1,73,1,0,2,0.0,0.0,...,8.0,14000.0,9.0,9.0,158,0.0,0,0,0,18000.0
3,26,1,N,1,73,1,0,2,0.0,0.0,...,7.0,14000.0,8.0,9.0,179,0.0,0,0,0,18000.0
4,26,2,N,2,66,1,0,2,0.0,0.0,...,6.0,7500.0,7.0,8.0,176,0.0,0,0,0,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75561,31,1,N,2,48,1,0,0,44.0,0.0,...,4.0,15000.0,8.0,5.0,148,30000.0,30000,10,0,30000.0
75562,31,2,N,2,21,1,0,2,0.0,0.0,...,5.0,15000.0,9.0,7.0,133,0.0,0,0,0,0.0
75563,31,2,N,2,21,1,0,2,0.0,0.0,...,6.0,17500.0,10.0,7.0,153,0.0,0,0,0,0.0
75564,31,2,N,2,22,1,0,2,0.0,0.0,...,6.0,22500.0,10.0,8.0,121,0.0,0,0,0,0.0


In [73]:
#Importamos la tabla de excel
tabla_adulto = pd.read_excel("tabla_adulto_equiv.xlsx", header = 3)
#Ajustamos el formato de la tabla 
tabla_adulto.dropna(axis = 1, inplace = True, thresh = 10)
tabla_adulto.dropna(inplace = True, thresh = 3)
tabla_adulto.reset_index(inplace = True, drop = True)
#Creamos la columna "adulto_equiv" como una columna de ceros
eph["adulto_equiv"] = 0
#Generamos una nueva columna llamada lista como una columna de ceros 
eph["lista"] = 0
#Iteramos por filas
#Para cada fila del data frame de acuerdo a la edad le indicamos que 
#la celda de la columna "lista" la complete con el número correspondiente 
#al índice de la tabla_adulto. 
#Por ejemplo, si la edad es menor a 30 entonces se asigna el numero 18 que es
#el índice de la fila de 18 a 29 años. 
for i in eph.index:
    if eph.loc[i, "CH06"] < 19:
        eph.loc[i, "lista"] = eph.loc[i, "CH06"]
    elif eph.loc[i, "CH06"] < 30:
        eph.loc[i, "lista"] = 18
    elif eph.loc[i, "CH06"] < 46:
        eph.loc[i, "lista"] = 19
    elif eph.loc[i, "CH06"] < 61:
        eph.loc[i, "lista"] = 20
    elif eph.loc[i, "CH06"] < 76:
        eph.loc[i, "lista"] = 21
    else:
        eph.loc[i, "lista"] = 22
#CH06 = edad, CH04 = Sexo (1 = varon y 2 = mujer)
#Para cada fila del dataframe si el individuo es varón entonces le indicamos que bajo 
#la columna "adulto_equiv" coloque la siguiente celda de tabla_adulto: 
# Fila: el índice que esta guardado en la columna "lista" en el df eph. 
# Columna: Varones 
#idem para las mujeres
#De este modo se obtienen los valores de adulto equivalente de cada persona según 
#su sexo y edad
for i in eph.index:
    if eph.loc[i, "CH04"] == 1:
        eph.loc[i, "adulto_equiv"] = tabla_adulto.loc[eph.loc[i, "lista"], "Varones"]
    elif eph.loc[i, "CH04"] == 2:
        eph.loc[i, "adulto_equiv"] = tabla_adulto.loc[eph.loc[i, "lista"], "Mujeres"]
#Eliminamos la columna "lista"
eph.drop("lista", axis = 1, inplace = True)
#Realizamos un groupby: para cada hogar sumamos "adulto_equiv"
ad_equiv_hogar = eph[["CODUSU","ANO4","TRIMESTRE","adulto_equiv"]].groupby(["CODUSU","ANO4","TRIMESTRE"]).sum()
ad_equiv_hogar.reset_index(inplace = True)
#cambiamos los nombres de las columnas
ad_equiv_hogar.columns = ["CODUSU","ANO4","TRIMESTRE","ad_equiv_hogar"]
#agregamos la columnna "ad_equiv_hogar" al df eph
eph = eph.merge(ad_equiv_hogar, on = ["CODUSU","ANO4","TRIMESTRE"], how = "left")

<ipython-input-73-7bbed948d211>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eph["adulto_equiv"] = 0
<ipython-input-73-7bbed948d211>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eph["lista"] = 0
C:\Users\felip\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [74]:
# Dividimos la muestra de la EPH en gente que no respondió ingresos totales familiares (ITF) y los que sí.
norespondieron = eph[eph["ITF"] == 0]
norespondieron.reset_index(inplace = True, drop = True)
respondieron = eph[eph["ITF"] != 0]
respondieron.reset_index(inplace = True, drop = True)

In [ ]:
# Ingresos mínimos por trimestre, hice un promedio del CBT por trimestre.
im_2t_2016 = 3812
im_3t_2016 = 4055
im_4t_2016 = 4232
im_1t_2017 = 4432
im_2t_2017 = 4744
im_3t_2017 = 4932
im_4t_2017 = 5219

# Creamos una columna de ingreso necesario para no ser pobre y la rellenamos con el for loop teniendo en cuenta el año
# y trimestre. Toma valores dependiendo de la columna ad_equiv_hogar * ingreso minimo per cápita.
respondieron["ingreso_necesario"] = 0
for i in respondieron.index:
    if (respondieron.loc[i, "ANO4"] == 2016) & (respondieron.loc[i, "TRIMESTRE"] == 2):
        respondieron.loc[i, "ingreso_necesario"] = im_2t_2016 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2016) & (respondieron.loc[i, "TRIMESTRE"] == 3):
        respondieron.loc[i, "ingreso_necesario"] = im_3t_2016 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2016) & (respondieron.loc[i, "TRIMESTRE"] == 4):
        respondieron.loc[i, "ingreso_necesario"] = im_4t_2016 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2017) & (respondieron.loc[i, "TRIMESTRE"] == 1):
        respondieron.loc[i, "ingreso_necesario"] = im_1t_2017 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2017) & (respondieron.loc[i, "TRIMESTRE"] == 2):
        respondieron.loc[i, "ingreso_necesario"] = im_2t_2017 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2017) & (respondieron.loc[i, "TRIMESTRE"] == 3):
        respondieron.loc[i, "ingreso_necesario"] = im_3t_2017 * respondieron.loc[i,"ad_equiv_hogar"]
    elif (respondieron.loc[i, "ANO4"] == 2017) & (respondieron.loc[i, "TRIMESTRE"] == 4):
        respondieron.loc[i, "ingreso_necesario"] = im_4t_2017 * respondieron.loc[i,"ad_equiv_hogar"]
        
# Creamos una columna dummy, que dice si el individuo es pobre o no. Depende de si sus ingresos
# son mayores a ingreso_necesario
respondieron["pobre"] = (respondieron["ITF"] < respondieron["ingreso_necesario"]).astype(int)

<ipython-input-75-8af44a3fadad>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  respondieron["ingreso_necesario"] = 0
C:\Users\felip\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
respondieron.pobre.sum()

In [ ]:
# Para arreglar que aparezcan si o si 4 veces, hago un count y si no aparecen 4 veces los elimino

In [25]:
arreglo = respondieron.groupby(["CODUSU","COMPONENTE"]).agg({"REGION":"count"})
arreglo[arreglo["REGION"]<4]

In [30]:
83120/4

20780.0

In [ ]:
# Preguntar que predecir, de que periodo a que.

In [ ]:
""" 
"AGLOMERADO", "COMPONENTE", "MAS_500", "CH04", "CH06", "CH08",
"CH09", "CH10", "CH11", "CH12", "CH13", "CH15", "CH07",
"CH16", "NIVEL_ED", "ESTADO", "CAT_OCUP", "CAT_INAC",
"PP02E", "PP02I", "PP3E_TOT", "PP3F_TOT", "PP04B3_ANO",
"PP03G", "PP03I", "INTENSI","PP11T", "PP11S",
"PP11O", "PP11N", "PP11L1", "PP10D", "PP10A", "PP10C",
"PP10E", "PP09C", "PP07K", "PP07H", "PP07G4", "PP11A",
"PP07G_59", "PP07G2", "PP07G1", "PP07C", "PP07A", "PP05H", "PP04G",
"TRIMESTRE", "CODUSU", "ANO4", "PONDERA", "IV1", "IV2", "IV11", "IV12_3",
"II7", "II8", "II9", "V5", "V17", "IV3", "V4", "IV6", "IV7", "IV8", "IV9",
"IV10", "IV11", "II4_1", "II4_2", "II4_3", "V1", "V2", "V3", "V6", "V8",
"V9", "V10", "V13", "V14", "V15", "V18", "IX_TOT", "IX_MEN10"
INGRESO: "ITF", "DECIFR", "ADECIFR", "IPCF", "DECCFR", "ADECCFR",
"PONDIH", "PP08D1", "P21", "DECOCUR", "TOT_P12", "P47T"
"""

In [40]:
columnas = ["AGLOMERADO","COMPONENTE","MAS_500","CH04","CH06","CH08","PP02E","PP02I","PP3E_TOT","PP3F_TOT",
"CH09","CH10","CH11","CH12","CH13","CH15","CH07","CH16","NIVEL_ED","ESTADO","CAT_OCUP","CAT_INAC","PP04B3_ANO",
"PP03G","PP03I","INTENSI","PP11T","PP11S","PP11O","PP11N","PP11L1","PP10D","PP10A","PP10C","PP07H","PP07G4","PP11A",
"PP10E","PP09C","PP07K","PP07G_59","PP07G2","PP07G1","PP07C","PP07A","PP05H","PP04G","IV2","IV11","IV12_3","IV1", 
"TRIMESTRE","CODUSU","ANO4","PONDERA","II7","II8","II9","V5","V17","IV3","V4","IV6","IV7","IV8","IV9","V3","V6","V8",
"IV10","IV11","II4_1","II4_2","II4_3","V1","V2","V9","V10","V13","V14","V15","V18","IX_TOT","IX_MEN10"]

In [43]:
columnas_ing = ["AGLOMERADO","COMPONENTE","MAS_500","CH04","CH06","CH08","PP02E","PP02I","PP3E_TOT","PP3F_TOT",
"CH09","CH10","CH11","CH12","CH13","CH15","CH07","CH16","NIVEL_ED","ESTADO","CAT_OCUP","CAT_INAC","PP04B3_ANO",
"PP03G","PP03I","INTENSI","PP11T","PP11S","PP11O","PP11N","PP11L1","PP10D","PP10A","PP10C","PP07H","PP07G4","PP11A",
"PP10E","PP09C","PP07K","PP07G_59","PP07G2","PP07G1","PP07C","PP07A","PP05H","PP04G","IV2","IV11","IV12_3","IV1", 
"TRIMESTRE","CODUSU","ANO4","PONDERA","II7","II8","II9","V5","V17","IV3","V4","IV6","IV7","IV8","IV9","V3","V6","V8",
"IV10","IV11","II4_1","II4_2","II4_3","V1","V2","V9","V10","V13","V14","V15","V18","IX_TOT","IX_MEN10","ITF","DECIFR",
"ADECIFR","IPCF","DECCFR","ADECCFR","PONDIH","PP08D1","P21","DECOCUR","TOT_P12","P47T"]

In [32]:
eph[eph["PP3E_TOT"]>100].PP3E_TOT.apply("count")

48